In [1]:
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score

import pysindy as ps
import pandas as pd

### Burgers equation

$$u_t = -uu_{x}+0.1u_{xx}$$

In [3]:
def load_burgers_data(path:str):
    
    burgers_data = loadmat(path)

    burgers_t = np.ravel(burgers_data["t"])
    burgers_x = np.ravel(burgers_data["x"])
    burgers_u = np.real(burgers_data["usol"])

    dt = burgers_t[1] - burgers_t[0]
    print(f"Sampling interval: {dt} seconds")
    print(f"x: {burgers_x.shape} t:{burgers_t.shape} u: {burgers_u.shape}")
    
    return (burgers_u, burgers_t, burgers_x)

def get_pdefind_library(spatial_grid):
    library_functions = [lambda x: x, lambda x: x * x]
    library_function_names = [lambda x: x, lambda x: x + x]

    pde_lib = ps.PDELibrary(
        library_functions=library_functions,
        function_names=library_function_names,
        derivative_order=3,
        spatial_grid=spatial_grid,
        is_uniform=True,    
        include_bias=True
    )    
    
    return pde_lib

def run_pdefind_burgers_exp():
    
    (burgers_u, burgers_t, burgers_x) = load_burgers_data("data/burgers.mat")
    pde_lib = get_pdefind_library(burgers_x)
    
    dt = burgers_t[1]-burgers_t[0]
    
    noises = [0.0, 0.001, 0.01, 0.1]
    burgers_u_dot = ps.FiniteDifference(axis=1)._differentiate(burgers_u, t=dt)

    for noise in noises:

        noise_mat = np.random.normal(0, noise, (len(burgers_x), len(burgers_t), 1))
        reshaped_burgers_u = burgers_u.reshape(len(burgers_x), len(burgers_t), 1)

        optimizer = ps.STLSQ(threshold=2, alpha=1e-5, normalize_columns=True)
        model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer, feature_names=["u"])
        model.fit(reshaped_burgers_u+noise_mat, t=dt)

        u_dot_predict = model.predict(reshaped_burgers_u)
        print(f'\nNoise: {noise} R2 Score: {r2_score(burgers_u_dot, u_dot_predict.reshape(u_dot_predict.shape[0], u_dot_predict.shape[1]))}')
        model.print()

run_pdefind_burgers_exp()

Sampling interval: 0.1 seconds
x: (256,) t:(101,) u: (256, 101)

Noise: 0.0 R2 Score: 0.9999838806846536
(u)' = 0.100 u_11 + -1.001 uu_1

Noise: 0.001 R2 Score: 0.9050044630048817
(u)' = 0.125 u + -0.352 uu + -0.259 u_1 + -0.654 uuu_11

Noise: 0.01 R2 Score: 0.8499567772604062
(u)' = 0.123 u + -0.345 uu + -1.166 uu_1 + -0.001 uu_11 + 0.509 uuu_11

Noise: 0.1 R2 Score: 0.25233637420156063
(u)' = -0.051 uu_1 + -0.287 uuu_11 + -0.001 uuu_111


### Weak PDE

In [30]:

def run_weaksindy_burgers_exp():
    
    (burgers_u, burgers_t, burgers_x) = load_burgers_data("data/burgers.mat")
    
    dt = burgers_t[1]-burgers_t[0]
    
    X, T = np.meshgrid(burgers_x, burgers_t)
    XT = np.asarray([X, T]).T
    
    library_functions = [lambda x: x, lambda x: x * x]
    library_function_names = [lambda x: x, lambda x: x + x]    
    
    pde_lib = ps.WeakPDELibrary(
        library_functions=library_functions,
        function_names=library_function_names,
        derivative_order=2,
        spatiotemporal_grid=XT,
        is_uniform=True,
        K = 1000
    )
    
    noises = [0.0, 0.001, 0.01, 0.1]
    burgers_u_dot = ps.FiniteDifference(axis=1)._differentiate(burgers_u, t=dt)

    for noise in noises:

        noise_mat = np.random.normal(0, noise, (len(burgers_x), len(burgers_t), 1))
        reshaped_burgers_u = burgers_u.reshape(len(burgers_x), len(burgers_t), 1)
        reshaped = burgers_u_dot.reshape(len(burgers_x), len(burgers_t))

        optimizer = ps.SR3(threshold=0.1, thresholder='l0',
                           tol=1e-8, normalize_columns=True, max_iter=1000)
        # optimizer = ps.STLSQ(threshold=0.1, alpha=1e-5, normalize_columns=True)
        
        model = ps.SINDy(feature_library=pde_lib, optimizer=optimizer,  feature_names=["u"])
        model.fit(reshaped_burgers_u+noise_mat, t=dt)
        model.print()    
        
#         pred = model.predict(reshaped_burgers_u)
#         print(burgers_u_dot.shape, reshaped_burgers_u.shape)
#         break

#         u_dot_predict = model.predict()        
#         print(u_dot_predict.shape, burgers_u_dot.shape)
#         score = r2_score(
#             burgers_u_dot, 
#             u_dot_predict.reshape(
#                 u_dot_predict.shape[0], u_dot_predict.shape[1]
#             )
#         )
        
#         print(f'\nNoise: {noise} R2 Score: {score}')
#         model.print()
        
run_weaksindy_burgers_exp()

Sampling interval: 0.1 seconds
x: (256,) t:(101,) u: (256, 101)
(u)' = 0.100 u_11 + -1.002 uu_1
(u)' = 0.100 u_11 + -1.002 uu_1
(u)' = 0.100 u_11 + -1.000 uu_1
(u)' = -0.954 uu_1
